In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re

In [2]:
# installeer webdriver indien nodig en maak benodigde URL driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://open.overheid.nl/zoekresultaten?text=woo%20verzoek&datumBeschikbaarVanaf=01-01-2019&datumBeschikbaarTot=31-12-2019&informatiesoort=c_4f50ca9c&page=1&count=2609")

# zet dataframe eerst als none zodat de columns toegevoegd kunnen worden
Dataframe = None

# de columnnamen zijn opzich ook te verzamelen door middel van Selenium, maar dat is wel onnodig.
column=['Titel', 'Verzoek', 'Specifiek', 'Verantwoordelijke', 'Thema', 'Documentsoort', 'Publicatiedatum', 'Documentdatum', 'Onderwerp']

# localiseer alle documenten
lists = driver.find_elements(By.CLASS_NAME, 'result--title ')

# vind alle URLS in de documenten
URLS = []
for i in lists:
    element = i.find_element(By.TAG_NAME, 'a').get_attribute("href")
    URLS.append(element)

# loop over al de URLS 
for i in URLS:
    # verander de driver met de nieuwe URL
    driver.get(i)
    
    # vind de titel van het document
    title = (driver.find_element(By.CLASS_NAME, "h2")).text
    
    # split titel in twee delen indien mogelijk
    split = title.split(" - ")
    if len(split) == 1:
        Row = [title, split[0], np.nan]
    else:
        Row = [title, split[0], split[1]]
    
    # vind de tabel met data en verzamel de data hieruit
    table = driver.find_element(By.CLASS_NAME, "table__data-overview")
    
    for row in table.find_elements(By.XPATH, ".//tr"):
        row_data = row.find_element(By.XPATH, ".//td").get_attribute('textContent')
        
        # als de data van een bepaald vak een lijst is maakt deze code er een python list van
        if ('\n') in row_data:
            Data = []
            
            i = row_data.split('\n')
            for item in i:
                res = re.findall(r'\w+', item)
                if res != []:
                    sentence = ' '.join([str(r) for r in res])
                    Data.append(sentence)
        else:
            Data = row_data
        
        Row.append(Data)
        
    # insert kolommmen en rijen in dataframe
    if Dataframe is None:
        Dataframe = pd.DataFrame(columns=column)

    if len(Row) == 8:
        Row.append(np.nan)
        
    Dataframe.loc[len(Dataframe)] = Row
    

driver.quit()

display(Dataframe)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 12.2MB/s]


,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp
0,Besluit op Wob-verzoek over Woo en Wbft,Besluit op Wob-verzoek over Woo en Wbft,NaN,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,31-01-2019,31-01-2019,NaN
1,Besluit Wob-verzoeken over een houthandelaar,Besluit Wob-verzoeken over een houthandelaar,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,30-01-2019,30-01-2019,Natuur en biodiversiteit
2,Besluit Wob-verzoek over verslag van Stuurgroe...,Besluit Wob-verzoek over verslag van Stuurgroe...,NaN,Ministerie van Infrastructuur en Waterstaat,Natuur en milieu,Beslissing op Wob-/Woo-verzoek,21-02-2019,21-02-2019,Luchtvaart
3,Besluit op Wob-verzoek over kennisgroep overdr...,Besluit op Wob-verzoek over kennisgroep overdr...,NaN,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,11-07-2019,10-07-2019,NaN
4,Besluit Wob-verzoek over Stoptober,Besluit Wob-verzoek over Stoptober,NaN,"Ministerie van Volksgezondheid, Welzijn en Sport",Gezondheidsrisico's,Beslissing op Wob-/Woo-verzoek,20-09-2019,27-03-2019,Roken
...,...,...,...,...,...,...,...,...,...
2604,Te verstrekken documenten,Te verstrekken documenten,NaN,Ministerie van Binnenlandse Zaken en Koninkrij...,Integratie,Beslissing op Wob-/Woo-verzoek,18-07-2019,18-07-2019,Gezichtsbedekkende kleding (in de media 'boerk...
2605,Besluit op Wob-verzoek over verrijkingsmateria...,Besluit op Wob-verzoek over verrijkingsmateria...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...","[Inrichting van de overheid, Landbouw visserij...",Beslissing op Wob-/Woo-verzoek,10-12-2019,09-12-2019,"[Wet openbaarheid van bestuur Wob, Veehouderij]"
2606,Besluit op Wob-verzoek over varkenshouders bed...,Besluit op Wob-verzoek over varkenshouders bed...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...","[Landbouw visserij voedselkwaliteit, Inrichtin...",Beslissing op Wob-/Woo-verzoek,04-11-2019,04-11-2019,"[Veehouderij, Wet openbaarheid van bestuur Wob]"
2607,Besluit op Wob-verzoek over trainingen diversi...,Besluit op Wob-verzoek over trainingen diversi...,NaN,Ministerie van Sociale Zaken en Werkgelegenheid,"[Rechten en vrijheden, Inrichting van de overh...",Beslissing op Wob-/Woo-verzoek,24-12-2019,17-10-2019,"[Discriminatie, Grondwet en Statuut, LHBTI ema..."


In [11]:
# Sla het op als csv file zodat dit niet nog een keer gerunt hoeft te worden. Duurde ongeveer 45 minuten namelijk
Dataframe.to_csv(r'C:\Users\ramon\Downloads\WOO2019.csv', index=False)

In [14]:
# laad de csv als dataframe
df = pd.read_csv('WOO2019.csv')
display(df)

,Titel,Verzoek,Specifiek,Verantwoordelijke,Thema,Documentsoort,Publicatiedatum,Documentdatum,Onderwerp
0,Besluit op Wob-verzoek over Woo en Wbft,Besluit op Wob-verzoek over Woo en Wbft,NaN,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,31-01-2019,31-01-2019,NaN
1,Besluit Wob-verzoeken over een houthandelaar,Besluit Wob-verzoeken over een houthandelaar,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...",Natuur- en landschapsbeheer,Beslissing op Wob-/Woo-verzoek,30-01-2019,30-01-2019,Natuur en biodiversiteit
2,Besluit Wob-verzoek over verslag van Stuurgroe...,Besluit Wob-verzoek over verslag van Stuurgroe...,NaN,Ministerie van Infrastructuur en Waterstaat,Natuur en milieu,Beslissing op Wob-/Woo-verzoek,21-02-2019,21-02-2019,Luchtvaart
3,Besluit op Wob-verzoek over kennisgroep overdr...,Besluit op Wob-verzoek over kennisgroep overdr...,NaN,Ministerie van Financiën,Onbekend thema,Beslissing op Wob-/Woo-verzoek,11-07-2019,10-07-2019,NaN
4,Besluit Wob-verzoek over Stoptober,Besluit Wob-verzoek over Stoptober,NaN,"Ministerie van Volksgezondheid, Welzijn en Sport",Gezondheidsrisico's,Beslissing op Wob-/Woo-verzoek,20-09-2019,27-03-2019,Roken
...,...,...,...,...,...,...,...,...,...
2604,Te verstrekken documenten,Te verstrekken documenten,NaN,Ministerie van Binnenlandse Zaken en Koninkrij...,Integratie,Beslissing op Wob-/Woo-verzoek,18-07-2019,18-07-2019,Gezichtsbedekkende kleding (in de media 'boerk...
2605,Besluit op Wob-verzoek over verrijkingsmateria...,Besluit op Wob-verzoek over verrijkingsmateria...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...","['Inrichting van de overheid', 'Landbouw visse...",Beslissing op Wob-/Woo-verzoek,10-12-2019,09-12-2019,"['Wet openbaarheid van bestuur Wob', 'Veehoude..."
2606,Besluit op Wob-verzoek over varkenshouders bed...,Besluit op Wob-verzoek over varkenshouders bed...,NaN,"Ministerie van Landbouw, Natuur en Voedselkwal...","['Landbouw visserij voedselkwaliteit', 'Inrich...",Beslissing op Wob-/Woo-verzoek,04-11-2019,04-11-2019,"['Veehouderij', 'Wet openbaarheid van bestuur ..."
2607,Besluit op Wob-verzoek over trainingen diversi...,Besluit op Wob-verzoek over trainingen diversi...,NaN,Ministerie van Sociale Zaken en Werkgelegenheid,"['Rechten en vrijheden', 'Inrichting van de ov...",Beslissing op Wob-/Woo-verzoek,24-12-2019,17-10-2019,"['Discriminatie', 'Grondwet en Statuut', 'LHBT..."


In [43]:
from collections import Counter

words = []

for i in df.Verzoek:
    i = i.split(' ', 1)
    words.append(i[0])
        
X = Counter(words).most_common()

print(X)

[('Besluit', 2053), ('Wob-verzoek', 124), ('Beslissing', 78), ('Bijlagen', 69), ('Bijlage', 38), ('Wob-besluit', 33), ('Deelbesluit', 28), ('Tweede', 26), ('Eerste', 22), ('1e', 18), ('Herzien', 16), ('2e', 16), ('Besluiten', 13), ('3e', 12), ('Bezwaar', 11), ('Wijzigingsbesluit', 9), ('Derde', 9), ('Documenten', 7), ('Hernieuwde', 6), ('Eeste', 4), ('Herziene', 4), ('Wob-verzoeken', 3), ('Twee', 3), ('Herstelbesluit', 1), ('Wob', 1), ('Inventarislijst', 1), ('JenV', 1), ('Nader', 1), ('Uitvoering', 1), ('Te', 1)]


In [47]:
titels = set()
for i in df.Titel:
    titels.add(i)
    
print('Titels:', len(titels))
    

besluiten = set()
for i in df.Verzoek:
    besluiten.add(i)
    
print("Besluitnamen",len(besluiten))

Titels: 1478
Besluitnamen 993
